# New evaluation codes

In [1]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from os.path import exists

sys.path.append('../..')

In [3]:
from stable_baselines3 import DQN

In [4]:
from experiments import preset_qcb_small
alpha = 0.25
beta = 0.75
extract = False
params, max_peaks = preset_qcb_small(None, alpha=alpha, beta=beta, extract_chromatograms=extract)
params, max_peaks

2022-08-22 22:19:37.778 | INFO     | experiments:get_samplers:269 - Loaded /Users/joewandy/Work/git/vimms-gym/pickles/samplers_QCB_small_gaussian.p


({'chemical_creator': {'mz_range': (100, 110),
   'rt_range': (400, 500),
   'intensity_range': (10000.0, 1e+20),
   'n_chemicals': (20, 50),
   'mz_sampler': <vimms.ChemicalSamplers.MZMLFormulaSampler at 0x7f7e149dc550>,
   'ri_sampler': <vimms.ChemicalSamplers.MZMLRTandIntensitySampler at 0x7f7e159a3d90>,
   'cr_sampler': <vimms.ChemicalSamplers.GaussianChromatogramSampler at 0x7f7e159c6730>},
  'noise': {'enable_spike_noise': True,
   'noise_density': 0.1,
   'noise_max_val': 1000.0,
   'mz_range': (100, 110)},
  'env': {'ionisation_mode': 'Positive',
   'rt_range': (400, 500),
   'isolation_window': 0.7,
   'mz_tol': 10,
   'rt_tol': 120,
   'alpha': 0.25,
   'beta': 0.75}},
 100)

In [5]:
from vimms_gym.env import DDAEnv
from stable_baselines3.common.monitor import Monitor
from vimms.Evaluation import EvaluationData
from vimms_gym.common import EVAL_METRIC_REWARD
from vimms_gym.evaluation import evaluate
from tune import TrialEvalCallback

import numpy as np
from timeit import default_timer as timer
from datetime import timedelta

In [6]:
custom_objects = {
    "learning_rate": 0.0,
    "lr_schedule": lambda _: 0.0,
    "clip_range": lambda _: 0.0,
}    

fname = os.path.join('..', 'DQN', 'DQN_24_rerun.zip') 
model = DQN.load(fname, custom_objects=custom_objects)

/opt/anaconda3/envs/vimms-gym/lib/python3.9/site-packages/stable_baselines3/common/buffers.py:561: UserWarning:

This system does not have apparently enough memory to store the complete replay buffer 0.99GB > 0.33GB



In [7]:
eval_env = DDAEnv(max_peaks, params)
eval_env

In [8]:
eval_env.env_params, eval_env.alpha, eval_env.beta

({'ionisation_mode': 'Positive',
  'rt_range': (400, 500),
  'isolation_window': 0.7,
  'mz_tol': 10,
  'rt_tol': 120,
  'alpha': 0.25,
  'beta': 0.75},
 0.25,
 0.75)

In [9]:
eval_env = Monitor(eval_env)
eval_env

<Monitor<DDAEnv instance>>

In [10]:
eval_metric = EVAL_METRIC_REWARD
eval_callback = TrialEvalCallback(eval_env, None, eval_metric)

In [11]:
n_eval_episodes = 10
verbose = 2

In [12]:
env = eval_callback.eval_env
env

In [13]:
deterministic = eval_callback.deterministic
deterministic

True

In [14]:
episode_count = 0
episode_count_target = n_eval_episodes
current_reward = 0
current_length = 0
observations = env.reset()
states = None
episode_starts = np.ones((env.num_envs,), dtype=bool)
episode_starts

array([ True])

In [15]:
episode_rewards = []
episode_eval_results = []
episode_lengths = []
start = timer()
while episode_count < episode_count_target:
    actions, states = model.predict(observations, state=states,
                                    episode_start=episode_starts,
                                    deterministic=deterministic)
    observations, rewards, dones, infos = env.step(actions)
    # print(rewards, current_reward, current_length, dones)
    episode_starts = dones
    current_reward += rewards[0]
    current_length += 1

    if dones[0]:  # when done, episode would be reset automatically
        val = current_reward
        eval_res = evaluate(eval_data, format_output=False)
        episode_eval_results.append(eval_res)
        if verbose > 0:
            end = timer()
            print('Evaluation episode %d finished: metric %f, timedelta=%s' % (
                episode_count, val, str(timedelta(seconds=end - start))))
            start = timer()
        episode_rewards.append(val)
        episode_lengths.append(current_length)
        episode_count += 1
        current_reward = 0
        current_length = 0

    # store previous results for evaluation before 'done'
    # this needs to be here, because VecEnv is automatically reset when done
    inner_env = env.envs[0].env
    eval_data = EvaluationData(inner_env.vimms_env)


Evaluation episode 0 finished: metric -2.182862, timedelta=0:00:01.088256
Evaluation episode 1 finished: metric 36.147723, timedelta=0:00:01.254650
Evaluation episode 2 finished: metric 34.005634, timedelta=0:00:01.233668
Evaluation episode 3 finished: metric 39.449035, timedelta=0:00:02.962317
Evaluation episode 4 finished: metric 43.804653, timedelta=0:00:01.853192
Evaluation episode 5 finished: metric -23.153187, timedelta=0:00:02.494667
Evaluation episode 6 finished: metric 36.833908, timedelta=0:00:01.211429
Evaluation episode 7 finished: metric 37.906845, timedelta=0:00:00.709591
Evaluation episode 8 finished: metric 38.923665, timedelta=0:00:02.315414
Evaluation episode 9 finished: metric 37.346098, timedelta=0:00:01.914113


In [16]:
episode_rewards

[-2.1828622548200656,
 36.14772261213511,
 34.005634096683934,
 39.44903507444542,
 43.80465339147486,
 -23.153186646290123,
 36.833908049506135,
 37.906845149816945,
 38.92366518967174,
 37.34609768680093]

In [17]:
np.mean(episode_rewards), np.std(episode_rewards)

(27.908151234942487, 20.961272215760548)

In [18]:
all_coverages = [e['coverage_prop'] for e in episode_eval_results]
np.mean(all_coverages), np.std(all_coverages)

(0.9728096903096903, 0.04192210423083383)

In [19]:
all_coverages = [e['intensity_prop'] for e in episode_eval_results]
np.mean(all_coverages), np.std(all_coverages)

(0.7857737253456435, 0.06854573154272864)